## Notebook used only to better visualize data processing

This is a visual aid to understand what is done with AFD dataframes in order to get registration data.

Actual code is done in a "normal" .py simply cause I don't like coding with multiple files/tables at once in notebooks.

This code might be outdated, but the overall process is this. 

In [2]:
import polars as pl
import mappers

In [6]:
# Function to iterate over each line and each column of the reduced dataframe to find new index values
# Since I'm 100% noobie with polars (I miss you, pandas!, but also not), this is chatGpt code :shrug: 
# I'm also 100% sure there's a better way to do this

def find_new_header_index_by_value(df, str_value): 
    registration_fields_line_index = None

    # Iterar pelas linhas do dataframe
    for i in range(df.shape[0]):  # range baseado no número de linhas
        if registration_fields_line_index: 
            break
        line = df[i].to_dict(as_series=False)  # Obtemos os dados da linha como dicionário
        for value in line.values():  # Itera sobre os valores da linha
            if str_value in str(value):  # Verifica se o valor procurado está presente
                registration_fields_line_index = i  # Se encontrado, armazena o índice da linha
                break  # Paramos a busca na linha atual após encontrar o valor

    return registration_fields_line_index

In [7]:
df = pl.read_parquet('../files/dataframe_parquet/Adequação da Formação Docente/2014.parquet')

Create a mini df with the first 10:10 registers, se we can easily look for a specific value that should be in the header. 

In [8]:
header_df = df.head(10).select(df.columns[:10])

Import a list with registration data fields and universal pk (school code)

In [9]:
registration_fields = mappers.registration_fields
universal_pk = mappers.universal_pk 

In [10]:
registration_fields_line_index = find_new_header_index_by_value(header_df, universal_pk)

In [11]:
# Usar a linha como novo cabeçalho
new_header = df.row(registration_fields_line_index)

In [12]:
# Redefinir os nomes das colunas e remover a linha usada como cabeçalho
df = df.rename({old: new for old, new in zip(df.columns, new_header) if new})

In [13]:
df = df.with_row_index().filter(~pl.col("index").is_in(registration_fields_line_index))

In [14]:
registration_dataframe = df.select([universal_pk] + registration_fields)

In [15]:
registration_dataframe

Código da Escola,Região,UF,Código do Município,Nome do Município,Nome da Escola,Localização,Dependência Administrativa
str,str,str,str,str,str,str,str
null,""" Instituto…",null,null,null,null,null,null
null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…
"""53088034""","""Centro_Oeste""","""DF""","""5300108""","""BRASILIA""","""OBRA SOCIAL NOSSA SENHORA DE F…","""Urbana""","""Privada"""
null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null


In [16]:
registration_dataframe = registration_dataframe.drop_nulls()

In [17]:
registration_dataframe

Código da Escola,Região,UF,Código do Município,Nome do Município,Nome da Escola,Localização,Dependência Administrativa
str,str,str,str,str,str,str,str
"""PK_COD_ENTIDADE""","""NO_REGIAO""","""SIGLA""","""PK_COD_MUNICIPIO""","""NO_MUNICIPIO""","""NO_ENTIDADE""","""TIPOLOCA""","""Dependad"""
"""11022558""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""EIEEF HAP BITT TUPARI""","""Rural""","""Estadual"""
"""11024275""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""CEEJA LUIZ VAZ DE CAMOES""","""Urbana""","""Estadual"""
"""11024291""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""EMMEF 07 DE SETEMBRO""","""Rural""","""Municipal"""
"""11024372""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""EMEIEF ANA NERY""","""Rural""","""Municipal"""
…,…,…,…,…,…,…,…
"""53085000""","""Centro_Oeste""","""DF""","""5300108""","""BRASILIA""","""COBIAN - COL BIANGULO""","""Urbana""","""Privada"""
"""53086007""","""Centro_Oeste""","""DF""","""5300108""","""BRASILIA""","""INSTEI - CE VICENTE PIRES""","""Urbana""","""Privada"""
"""53087003""","""Centro_Oeste""","""DF""","""5300108""","""BRASILIA""","""CED SANTOS DUMONT - CESAN""","""Urbana""","""Privada"""


In [18]:
registration_dataframe = registration_dataframe.filter(pl.col(universal_pk).cast(pl.Int32, strict=False).is_not_null())

In [19]:
registration_dataframe

Código da Escola,Região,UF,Código do Município,Nome do Município,Nome da Escola,Localização,Dependência Administrativa
str,str,str,str,str,str,str,str
"""11022558""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""EIEEF HAP BITT TUPARI""","""Rural""","""Estadual"""
"""11024275""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""CEEJA LUIZ VAZ DE CAMOES""","""Urbana""","""Estadual"""
"""11024291""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""EMMEF 07 DE SETEMBRO""","""Rural""","""Municipal"""
"""11024372""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""EMEIEF ANA NERY""","""Rural""","""Municipal"""
"""11024593""","""Norte""","""RO""","""1100015""","""ALTA FLORESTA D OESTE""","""EMEIEF DARCY PENTEADO""","""Rural""","""Municipal"""
…,…,…,…,…,…,…,…
"""53085000""","""Centro_Oeste""","""DF""","""5300108""","""BRASILIA""","""COBIAN - COL BIANGULO""","""Urbana""","""Privada"""
"""53086007""","""Centro_Oeste""","""DF""","""5300108""","""BRASILIA""","""INSTEI - CE VICENTE PIRES""","""Urbana""","""Privada"""
"""53087003""","""Centro_Oeste""","""DF""","""5300108""","""BRASILIA""","""CED SANTOS DUMONT - CESAN""","""Urbana""","""Privada"""
